In [1]:
import pandas as pd
import numpy as np
from ChainLadder import ChainLadder
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
# 20191010: 지급보험금 진전추이 → 지급보험금 + 개별O/S 진전추이

In [2]:
pd.options.display.float_format = '{:,.4f}'.format

In [3]:
일반출재계약 = pd.read_excel('./data/일반_출재_계약_20190812.xlsx', dtype={'UY': str, 'FY': str, 'AY': str, 'RRNR_DAT_DVCD': str})
일반출재보상 = pd.read_excel('./data/일반_출재_보상_20190812.xlsx', dtype={'UY': str, 'FY': str, 'AY': str, 'RRNR_DAT_DVCD': str, 'RN_PRM': float, 'RN_ELP_PRM': float })
준비금위험계수 = pd.read_excel('./data/충격시나리오_기초정보.xlsx', sheet_name='준비금위험계수')
특약별_정산수수료 = pd.read_excel('./data/특약별_정산수수료.xlsx', dtype={'UY': str})
일반특약정보 = pd.read_excel('./data/일반_특약정보_20190811.xlsx')

In [6]:
일반출재보상_가공 = 일반출재보상.query('AY != " "')
일반출재보상_가공 = 일반출재보상_가공.loc[~일반출재보상_가공['AY'].isna()]
일반출재보상_가공 = 일반출재보상_가공.loc[~일반출재보상_가공['UY'].isna()]
일반출재보상_가공['DY'] = (일반출재보상_가공['FY'].astype(int) - 일반출재보상_가공['AY'].astype(int) + 1)
일반출재보상_가공 = 일반출재보상_가공.query('DY >= 1')
일반출재보상_가공 = 일반출재보상_가공.query('RRNR_DAT_DVCD != "04"')
일반출재보상_가공 = 일반출재보상_가공.groupby(['NTNL_CTRY_CD', 'PDGR_CD', 'DY', 'AY'])[['RN_OGL_BNF', 'RN_OST_AMT']].sum().reset_index()
일반출재보상_가공['RN_LOSS'] = 일반출재보상_가공['RN_OGL_BNF'] + 일반출재보상_가공['RN_OST_AMT']

In [7]:
기준년도 = '2018'

result = []
for 상품군코드 in 준비금위험계수['보종코드']:

    n = 7 if 상품군코드 in (25, 26) else 5
    시작년도 = str(int(기준년도) - n + 1)

    # 삼각형 생성
    일반출재보상_보종 = 일반출재보상_가공.query('NTNL_CTRY_CD == "KR"').query('PDGR_CD == @상품군코드')
    일반출재보상_보종 = 일반출재보상_보종.query('AY >= @시작년도').query('DY <= @n')
    
    삼각형_보종 = 일반출재보상_보종.pivot_table(index='AY', columns='DY', values='RN_LOSS', aggfunc=np.sum)
    삼각형_보종.index.name = 'Accident'
    삼각형_보종.columns.name = 'Development'


    일반출재계약_가공 = 일반출재계약.query('RRNR_DAT_DVCD != "04"').groupby(['NTNL_CTRY_CD', 'PDGR_CD', 'FY'])[['RN_ELP_PRM']].sum().reset_index()
    일반출재계약_보종 = 일반출재계약_가공.query('NTNL_CTRY_CD == "KR"').query('PDGR_CD == @상품군코드')
    일반출재계약_보종 = 일반출재계약_보종.query('FY >= @시작년도')

    cl = ChainLadder(삼각형_보종, type='incr')
    
    위험계수 = 준비금위험계수.loc[준비금위험계수['보종코드'] == 상품군코드, '준비금위험계수'].values[0]
    최종손해액_준비금충격_보종 = cl.reserve(method='weighted average')*(1+위험계수) + cl.triangle_incr.fillna(0).sum(axis=1).to_frame('Reserve')
    최종손해율_준비금충격_보종 = 최종손해액_준비금충격_보종/일반출재계약_보종['RN_ELP_PRM'].values.reshape(-1, 1)
    최종손해액_보종 = cl.reserve(method='weighted average') + cl.triangle_incr.fillna(0).sum(axis=1).to_frame('Reserve')
    최종손해율_보종 = 최종손해액_보종/일반출재계약_보종['RN_ELP_PRM'].values.reshape(-1, 1)
    추가진전율_보종 = 최종손해율_준비금충격_보종-최종손해율_보종
    추가진전율_보종.insert(0, 'PDGR_CD', 상품군코드)

    result.append(추가진전율_보종)
df = pd.concat(result).reset_index()
df.columns = ['사고년도', '보종코드', '추가진전율']

In [8]:
특약명_전체 = ['재물보험특약', '기술보험_일반', '기술보험_ALOP', '근재보험특약', '배상책임보험특약', '해외PST']
result = []
for 특약명 in 특약명_전체:

    산출기간 = 7 if 특약명 == '근재보험특약' or 특약명 == '배상책임보험특약' else 5
    UY_전체 = np.arange(int(기준년도)-산출기간+1, int(기준년도)+1).astype(str)
    
    for UY in UY_전체:

        특약코드 = 일반특약정보.loc[일반특약정보['TTY_CD_NM'].str.contains(특약명), 'RRNR_TTY_CD']
        특약코드_UY = 특약코드.loc[특약코드.str.contains('U{}'.format(UY))]

        출재계약_특약_UY = 일반출재계약.query('RRNR_TTY_CD in @특약코드_UY').query('UY == @UY')
        출재보상_특약_UY = 일반출재보상.query('RRNR_TTY_CD in @특약코드_UY').query('UY == @UY')
        보종코드목록 = 출재계약_특약_UY.PDGR_CD.drop_duplicates()

        for 보종코드 in 보종코드목록:

            # 출재 계약 (경과보험료)
            출재계약_특약_UY_보종 = 출재계약_특약_UY.query('PDGR_CD == @보종코드')
            출재보상_특약_UY_보종 = 출재보상_특약_UY.query('PDGR_CD == @보종코드')
            출재경과보험료_UY_국내_원수_보종 = 일반출재계약.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['RN_ELP_PRM'].sum()
            출재경과보험료_특약_UY_보종 = 출재계약_특약_UY_보종['RN_ELP_PRM'].sum()

            # 출재 보상 (보험금)
            출재보험금_UY_국내_원수_보종 = 일반출재보상.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['RN_OGL_BNF'].sum()
            출재보험금_특약_UY_보종 = 출재보상_특약_UY_보종['RN_OGL_BNF'].sum()

            # 출재 보상 (준비금)
            출재준비금_UY_국내_원수_보종 = 일반출재보상.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['RN_OST_AMT'].sum()
            출재준비금_특약_UY_보종 = 출재보상_특약_UY_보종['RN_OST_AMT'].sum()

            # 출재 보상 (손해조사비)
            출재손조비_UY_국내_원수_보종 = 일반출재보상.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['WNCR_CNV_RN_DAG_IVMT'].sum()
            출재손조비_특약_UY_보종 = 출재보상_특약_UY_보종['WNCR_CNV_RN_DAG_IVMT'].sum()

            # 출재 보상 (손해액)
            출재손해액_특약_UY_보종 = 출재보험금_특약_UY_보종 + 출재준비금_특약_UY_보종 + 출재손조비_특약_UY_보종

            # 출재 보상 (손해율)
            출재손해율_특약_UY_보종 = 출재손해액_특약_UY_보종/출재경과보험료_특약_UY_보종
            출재손해율_특약_UY_보종

            컬럼 = ['특약명', 'UY', '보종코드', '출재경과보험료', '출재손해율']
            데이터 = [특약명, UY, 보종코드, 출재경과보험료_특약_UY_보종, 출재손해율_특약_UY_보종]
            dg = pd.DataFrame(데이터, 컬럼).T
            result.append(dg)
dg = pd.concat(result)

In [9]:
df2 = dg.merge(df, left_on=['UY', '보종코드'], right_on=['사고년도', '보종코드'], how='left')
df2 = df2.dropna(axis=0) # 근재보험특약/종합보종은 5년까지 밖에 없음
df2['출재손해율_충격'] = df2['출재손해율'] + df2['추가진전율']
df2 = df2.merge(특약별_정산수수료, on=['특약명', 'UY'], how='left').drop(['정산식', '사고년도'], axis=1)
df2['수수료_실적기준'] = df2['출재경과보험료']*np.fmax(np.fmin(df2['Max'], df2['기울기']*(df2['a']-df2['출재손해율'])+df2['b']), df2['Min'])
df2['수수료_충격기준'] = df2['출재경과보험료']*np.fmax(np.fmin(df2['Max'], df2['기울기']*(df2['a']-df2['출재손해율_충격'])+df2['b']), df2['Min'])
df2['수수료증감'] = df2['수수료_충격기준'] - df2['수수료_실적기준']

In [10]:
df3 = df2.groupby(['특약명', '보종코드'])[['수수료증감']].sum().reset_index()
df3

,특약명,보종코드,수수료증감
0,근재보험특약,25,"-48,715,902.7901"
1,기술보험_ALOP,24,0.0000
2,기술보험_일반,24,"1,382,505.1237"
3,기술보험_일반,27,0.0000
4,배상책임보험특약,26,"-517,985,127.5618"
5,배상책임보험특약,27,"4,148,889.0698"
6,재물보험특약,23,"129,579,693.4733"
7,재물보험특약,27,"121,639,753.8335"
8,해외PST,24,-319.0435
9,해외PST,27,"74,819,663.7007"


In [11]:
with pd.ExcelWriter('./result/연동수수료특약_충격시나리오법_준비금_{}.xlsx'.format(datetime.now().strftime('%Y%m%d%H%M%S'))) as writer:
    df2.to_excel(writer, index=False)